# Decorators

See PEPs [318](https://www.python.org/dev/peps/pep-0318/) and [3129](https://www.python.org/dev/peps/pep-3129/)

In [1]:
def foo(x):
    print('Preprocess')
    print('Execute foo: ', x)

foo(7)

Preprocess
Execute foo:  7


In [2]:
def bar(x):
    print('Preprocess')
    print('Execute bar: ', x)

bar(42)

Preprocess
Execute bar:  42


**DRY!**

In [4]:
# We already know that a function can be passed, defined in another function or returned...
def preprocess(f):
    def wrap(x):
        print('Preprocess')
        f(x)
    return wrap

In [5]:
def foo2(x):
    print('Execute foo: ', x)

pfoo2 = preprocess(foo2)
pfoo2(7)

Preprocess
Execute foo:  7


In [6]:
def bar2(x):
    print('Execute bar: ', x)

pbar2 = preprocess(bar2)
pbar2(42)

Preprocess
Execute bar:  42


Or...

In [7]:
@preprocess
def foo3(x):
    print('Execute foo: ', x)
    
foo3(7)

Preprocess
Execute foo:  7


In [8]:
@preprocess
def bar3(x):
    print('Execute bar: ', x)

bar3(42)

Preprocess
Execute bar:  42


# Common Python Decorator Patterns

- Wrapper layer for simple timing

In [9]:
import time
from functools import wraps

def awesometimer(f):
    '''Decorator that times a function f'''
    @wraps(f)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        print(f.__name__ + ':', end - start)
        return result
    return wrapper

@awesometimer
def foo(x):
    '''Some computationally intensive function'''
    while x % 10000000 != 0:
        x += 1

foo(1)

foo: 1.2910740375518799


- You want to write a logger which, let's assume involves passing args to a decorator

In [10]:
import logging
from functools import wraps

def logged(level, name=None, msg=None):
    '''Adds logging'''
    def decorate(f):
        logname = name if name else f.__module__
        log = logging.getLogger(logname)
        logmsg = msg if msg else f.__name__
        
        @wraps(f)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return f(*args, **kwargs)
        return wrapper
    return decorate

@logged(logging.CRITICAL, name='ohgodno', msg='Oh God! Nooo!')
def ohgodno():
    print('Executing ohgodno')
    
ohgodno()

CRITICAL:ohgodno:Oh God! Nooo!


Executing ohgodno


# Class as a decorator

In [12]:
class ComplexProcessor(object):
    """Amazing decorator"""
    def __init__(self, f):
        self.f, self.complex_flag = f, 0
        wraps(f)(self)
    
    def complex_method(self):
        """complex processing"""
        self.complex_flag = 5

    def __call__(self, *args, **kwargs):
        """magic happens here"""
        print('State: ', self.complex_flag)
        print('Begin complex processing...')
        self.complex_method()
        print('End complex processing')
        print('State: ', self.complex_flag)
        response = self.f(*args, **kwargs)
        print('Post execution cleanup')
        return response


@ComplexProcessor
def foo4():
    """Some repeated function pattern"""
    print('foo4')

foo4()

State:  0
Begin complex processing...
End complex processing
State:  5
foo4
Post execution cleanup
